In [ ]:
!pip install itslive
#conda install -c conda-forge itslive

In [ ]:
import numpy as np
import pandas as pd
# import geopandas as gpd
import os
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates
import itslive
import pyproj
import seaborn as sns
import colorcet as cc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#extract points (lon, lat) from a CSV
site_abbrev = 'ASG'
root_dir = '/content/drive/MyDrive/Greenland_FWF/personal_workspaces/ellyn'
transect_inc = '1000' #transect spacing (meters)

#read csv from computer
pts = pd.read_csv(root_dir+'/'+site_abbrev+'/shapefiles/'+site_abbrev+'-centerline_'+transect_inc+'m-interval.csv')
pts_list = list(zip(pts['Easting (m)'],pts['Northing (m)']))
# print(pts_list)

# reproject
source_crs = pyproj.CRS("EPSG:3413")  # WGS84 = EPSG : '4326' (Lat,Lon), Greenland PS = EPSG : '3413' (Northing, Easting)
target_crs = pyproj.CRS("EPSG: 4326")
transform_func = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
coords_list = [None] * len(pts_list)
for i in range(len(pts_list)):
  lon,lat = transform_func.transform(pts_list[i][0],pts_list[i][1])
  coords_list[i] = (lon,lat)

# coords_list

In [ ]:
#extract and save itslive velocity timeseries at each point

#check that there is a "velocities" directory and create it as necessary
if not os.path.exists(root_dir+'/'+site_abbrev+'/velocities'):
  os.makedirs(root_dir+'/'+site_abbrev+'/velocities')

#create a dataframe with time and velocity for each point (1 starts in the ocean)
for i in range(len(coords_list)):
  pt = coords_list[i]
  timeseries_datasets = itslive.velocity_cubes.get_time_series([pt])
  #velocity vector
  velocity = timeseries_datasets[0]['time_series']['v']
  velocity = velocity.fillna(0) #replace NaNs with zeros (for saving to CSV)
  #mid date for the velocity estimate (as a datetime)
  mid_date = timeseries_datasets[0]['time_series']['mid_date']
  mid_date = pd.to_datetime(mid_date)
  #time separate between images for velocity estimate (days only)
  date_dt = timeseries_datasets[0]['time_series']['date_dt']
  days_only = date_dt.astype('timedelta64[D]')
  ddays = days_only / np.timedelta64(1, 'D')

  #save
  formatted_index = f"{i+1:02}"
  print(formatted_index)
  v = pd.DataFrame({'mid_date':mid_date,'days_dt':ddays,'velocity (m/yr)':velocity})
  v.to_csv(root_dir+'/'+site_abbrev+'/velocities/itslive_velocity_transect' + formatted_index + '.csv', index=False)

print('Velocities extracted!')
# timeseries_datasets

**Visualize data in one of two ways: (1 = top cell) for one point at a time all within the loop or (2 = subsequent cells) for all the data together:**

In [ ]:
min_speed = 100; #threshold value for reasonable speeds (m/yr)

In [ ]:
## Option 2) load the data for all points all at once & plot in a single figure
pt_dict = {}
for i in range(len(coords_list)):
  formatted_index = f"{i+1:02}"
  #name variables based on index within the loop
  pt_dict[f"pt{i}"] = pd.read_csv(root_dir+'/'+site_abbrev+'/velocities/itslive_velocity_transect'+formatted_index+'.csv')
  pt_dict[f"pt{i}"]['mid_datetime'] = pd.to_datetime(pt_dict[f"pt{i}"]['mid_date'])
# print(pt_dict) #example to output full dictionary
# print(pt_dict["pt3"]) #example to call a specific index

# create a list of colors
# colors = sns.color_palette("icefire", len(pt_dict))
colors = sns.color_palette(cc.glasbey, n_colors=len(pt_dict))

#create seperate plots for velocity over time at each point
plt.figure(figsize=(20, 4))
for i in range(len(pt_dict)):
  df = pt_dict[f"pt{i}"]

  # mask-out clearly incorrect velocities
  df['velocity (m/yr)'] = df['velocity (m/yr)'].replace(0, np.nan) #replace zeros with NaNs
  df['velocity (m/yr)'] = df['velocity (m/yr)'].mask(df['velocity (m/yr)'] < min_speed, np.nan) #replace values below a specific threshold with NaNs

  # filter data by days_dt
  df = df[df['days_dt'] < 60]
  df = df[df['days_dt'] > 15]

  #rename
  velocity_middate = df['mid_datetime'].values
  velocity_dt = df['days_dt'].values
  velocity_data = df['velocity (m/yr)'].values

  # add scatterplot to figure with a specified color
  plt.scatter(velocity_middate, velocity_data,color=colors[i % len(colors)])  # Plot the data

plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
# plt.title(site_abbrev+' velocity pt ' + formatted_index)
plt.title(site_abbrev)
plt.xlim(datetime(2013, 1, 1), datetime(2026, 1, 1))
# plt.ylim(-50, 1000)
plt.grid(True)
plt.legend(labels=[f"pt{i+1}" for i in range(len(pt_dict))], loc='right', ncol=2)
plt.show()

In [ ]:
## Option 2) one point at a time, each as a different plot

#read in each csv
for i in range(len(coords_list)):
  formatted_index = f"{i+1:02}"
  pt = pd.read_csv(root_dir+'/'+site_abbrev+'/velocities/itslive_velocity_transect' + formatted_index + '.csv')
  pt['mid_datetime'] = pd.to_datetime(pt['mid_date']) #convert to datetime (useful later for sorting)
  df = pt #rename the dataframe for consistency with the option below

  #mask-out clearly incorrect velocities
  df['velocity (m/yr)'] = df['velocity (m/yr)'].replace(0, np.nan) #replace zeros with NaNs
  df['velocity (m/yr)'] = df['velocity (m/yr)'].mask(df['velocity (m/yr)'] < min_speed, np.nan) #replace values below a specific threshold with NaNs

  #plot the data
  velocity_middate = df['mid_datetime'].values
  velocity_data = df['velocity (m/yr)'].values
  plt.figure(figsize=(20, 4))
  plt.scatter(velocity_middate, velocity_data)  # Plot the data
  plt.xlabel('Time')
  plt.ylabel('Velocity (m/yr)')
  plt.title(site_abbrev+' velocity pt ' + formatted_index)
  plt.xlim(datetime(2013, 1, 1), datetime(2025, 1, 1))
  # plt.ylim(-50, 1000)
  plt.show()


**Scratch code to hone in on each individual year:**

winter 2015/2016:

In [ ]:
starttime = datetime(2015, 11, 1)
endttime = datetime(2016, 2, 1)

v_winter_pt1_2015 = pt1[(pt1['mid_date'] >= starttime) & (pt1['mid_date'] <= endttime)]
v_winter_pt2_2015 = pt2[(pt2['mid_date'] >= starttime) & (pt2['mid_date'] <= endttime)]
v_winter_pt3_2015 = pt3[(pt3['mid_date'] >= starttime) & (pt3['mid_date'] <= endttime)]
v_winter_pt4_2015 = pt4[(pt4['mid_date'] >= starttime) & (pt4['mid_date'] <= endttime)]
v_winter_pt5_2015 = pt5[(pt5['mid_date'] >= starttime) & (pt5['mid_date'] <= endttime)]
v_winter_pt6_2015 = pt6[(pt6['mid_date'] >= starttime) & (pt6['mid_date'] <= endttime)]
# v_winter_pt7_2015 = pt7[(pt7['mid_date'] >= starttime) & (pt7['mid_date'] <= endttime)]
# v_winter_pt8_2015 = pt8[(pt8['mid_date'] >= starttime) & (pt8['mid_date'] <= endttime)]

In [ ]:
#df of winter velocities at each point from november-february 2015
v_winter_2015 = pd.concat([v_winter_pt1_2015, v_winter_pt2_2015, v_winter_pt3_2015, v_winter_pt4_2015, v_winter_pt5_2015, v_winter_pt6_2015])

#plot
velocity_winter_2015 = v_winter_2015['velocity (m/yr)'].values
time_winter_2015 = pd.to_datetime(v_winter_2015['mid_date']).values
plt.figure(figsize=(20, 4))
plt.scatter(time_winter_2015, velocity_winter_2015)
plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
plt.title('Velocity Winter 2015')
plt.ylim(-50, 600)
plt.show()

winter 2016/2017:

In [ ]:
starttime = datetime(2016, 11, 1)
endttime = datetime(2017, 2, 1)

v_winter_pt1_2016 = pt1[(pt1['mid_date'] >= starttime) & (pt1['mid_date'] <= endttime)]
v_winter_pt2_2016 = pt2[(pt2['mid_date'] >= starttime) & (pt2['mid_date'] <= endttime)]
v_winter_pt3_2016 = pt3[(pt3['mid_date'] >= starttime) & (pt3['mid_date'] <= endttime)]
v_winter_pt4_2016 = pt4[(pt4['mid_date'] >= starttime) & (pt4['mid_date'] <= endttime)]
v_winter_pt5_2016 = pt5[(pt5['mid_date'] >= starttime) & (pt5['mid_date'] <= endttime)]
v_winter_pt6_2016 = pt6[(pt6['mid_date'] >= starttime) & (pt6['mid_date'] <= endttime)]
# v_winter_pt7_2016 = pt7[(pt7['mid_date'] >= starttime) & (pt7['mid_date'] <= endttime)]
# v_winter_pt8_2016 = pt8[(pt8['mid_date'] >= starttime) & (pt8['mid_date'] <= endttime)]

In [ ]:
#df of winter velocities at each point from november-february 2016/2017
v_winter_2016 = pd.concat([v_winter_pt1_2016, v_winter_pt2_2016, v_winter_pt3_2016, v_winter_pt4_2016, v_winter_pt5_2016, v_winter_pt6_2016])

#plot
velocity_winter_2016 = v_winter_2016['velocity (m/yr)'].values
time_winter_2016 = pd.to_datetime(v_winter_2016['mid_date']).values
plt.figure(figsize=(20, 4))
plt.scatter(time_winter_2016, velocity_winter_2016)
plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
plt.title('Velocity Winter 2016')
plt.ylim(-50, 600)
plt.show()

winter 2017/2018:

In [ ]:
starttime = datetime(2017, 11, 1)
endttime = datetime(2018, 2, 1)

v_winter_pt1_2017 = pt1[(pt1['mid_date'] >= starttime) & (pt1['mid_date'] <= endttime)]
v_winter_pt2_2017 = pt2[(pt2['mid_date'] >= starttime) & (pt2['mid_date'] <= endttime)]
v_winter_pt3_2017 = pt3[(pt3['mid_date'] >= starttime) & (pt3['mid_date'] <= endttime)]
v_winter_pt4_2017 = pt4[(pt4['mid_date'] >= starttime) & (pt4['mid_date'] <= endttime)]
v_winter_pt5_2017 = pt5[(pt5['mid_date'] >= starttime) & (pt5['mid_date'] <= endttime)]
v_winter_pt6_2017 = pt6[(pt6['mid_date'] >= starttime) & (pt6['mid_date'] <= endttime)]
# v_winter_pt7_2017 = pt7[(pt7['mid_date'] >= starttime) & (pt7['mid_date'] <= endttime)]
# v_winter_pt8_2017 = pt8[(pt8['mid_date'] >= starttime) & (pt8['mid_date'] <= endttime)]

In [ ]:
#df of winter velocities at each point from november-february 2017/2018
v_winter_2017 = pd.concat([v_winter_pt1_2017, v_winter_pt2_2017, v_winter_pt3_2017, v_winter_pt4_2017, v_winter_pt5_2017, v_winter_pt6_2017])

#plot
velocity_winter_2017 = v_winter_2017['velocity (m/yr)'].values
time_winter_2017 = pd.to_datetime(v_winter_2017['mid_date']).values
plt.figure(figsize=(20, 4))
plt.scatter(time_winter_2017, velocity_winter_2017)
plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
plt.title('Velocity Winter 2017')
plt.ylim(-50, 600)
plt.show()

winter 2018/2019:

In [ ]:
starttime = datetime(2018, 11, 1)
endttime = datetime(2019, 2, 1)

v_winter_pt1_2018 = pt1[(pt1['mid_date'] >= starttime) & (pt1['mid_date'] <= endttime)]
v_winter_pt2_2018 = pt2[(pt2['mid_date'] >= starttime) & (pt2['mid_date'] <= endttime)]
v_winter_pt3_2018 = pt3[(pt3['mid_date'] >= starttime) & (pt3['mid_date'] <= endttime)]
v_winter_pt4_2018 = pt4[(pt4['mid_date'] >= starttime) & (pt4['mid_date'] <= endttime)]
v_winter_pt5_2018 = pt5[(pt5['mid_date'] >= starttime) & (pt5['mid_date'] <= endttime)]
v_winter_pt6_2018 = pt6[(pt6['mid_date'] >= starttime) & (pt6['mid_date'] <= endttime)]
# v_winter_pt7_2018 = pt7[(pt7['mid_date'] >= starttime) & (pt7['mid_date'] <= endttime)]
# v_winter_pt8_2018 = pt8[(pt8['mid_date'] >= starttime) & (pt8['mid_date'] <= endttime)]

In [ ]:
#df of winter velocities at each point from november-february 2018/2019
v_winter_2018 = pd.concat([v_winter_pt1_2018, v_winter_pt2_2018, v_winter_pt3_2018, v_winter_pt4_2018, v_winter_pt5_2018, v_winter_pt6_2018])

#plot
velocity_winter_2018 = v_winter_2018['velocity (m/yr)'].values
time_winter_2018 = pd.to_datetime(v_winter_2018['mid_date']).values
plt.figure(figsize=(20, 4))
plt.scatter(time_winter_2018, velocity_winter_2018)
plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
plt.title('Velocity Winter 2018')
plt.ylim(-50, 600)
plt.show()

winter 2019/2020:

In [ ]:
starttime = datetime(2019, 11, 1)
endttime = datetime(2020, 2, 1)

v_winter_pt1_2019 = pt1[(pt1['mid_date'] >= starttime) & (pt1['mid_date'] <= endttime)]
v_winter_pt2_2019 = pt2[(pt2['mid_date'] >= starttime) & (pt2['mid_date'] <= endttime)]
v_winter_pt3_2019 = pt3[(pt3['mid_date'] >= starttime) & (pt3['mid_date'] <= endttime)]
v_winter_pt4_2019 = pt4[(pt4['mid_date'] >= starttime) & (pt4['mid_date'] <= endttime)]
v_winter_pt5_2019 = pt5[(pt5['mid_date'] >= starttime) & (pt5['mid_date'] <= endttime)]
v_winter_pt6_2019 = pt6[(pt6['mid_date'] >= starttime) & (pt6['mid_date'] <= endttime)]
# v_winter_pt7_2019 = pt7[(pt7['mid_date'] >= starttime) & (pt7['mid_date'] <= endttime)]
# v_winter_pt8_2019 = pt8[(pt8['mid_date'] >= starttime) & (pt8['mid_date'] <= endttime)]

In [ ]:
#df of winter velocities at each point from november-february 2019/2020
v_winter_2019 = pd.concat([v_winter_pt1_2019, v_winter_pt2_2019, v_winter_pt3_2019, v_winter_pt4_2019, v_winter_pt5_2019, v_winter_pt6_2019])

#plot
velocity_winter_2019 = v_winter_2019['velocity (m/yr)'].values
time_winter_2019 = pd.to_datetime(v_winter_2019['mid_date']).values
plt.figure(figsize=(20, 4))
plt.scatter(time_winter_2019, velocity_winter_2019)
plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
plt.title('Velocity Winter 2019')
plt.ylim(-50, 600)
plt.show()

winter 2020/2021

In [ ]:
starttime = datetime(2020, 11, 1)
endttime = datetime(2021, 2, 1)

v_winter_pt1_2020 = pt1[(pt1['mid_date'] >= starttime) & (pt1['mid_date'] <= endttime)]
v_winter_pt2_2020 = pt2[(pt2['mid_date'] >= starttime) & (pt2['mid_date'] <= endttime)]
v_winter_pt3_2020 = pt3[(pt3['mid_date'] >= starttime) & (pt3['mid_date'] <= endttime)]
v_winter_pt4_2020 = pt4[(pt4['mid_date'] >= starttime) & (pt4['mid_date'] <= endttime)]
v_winter_pt5_2020 = pt5[(pt5['mid_date'] >= starttime) & (pt5['mid_date'] <= endttime)]
v_winter_pt6_2020 = pt6[(pt6['mid_date'] >= starttime) & (pt6['mid_date'] <= endttime)]
v_winter_pt7_2020 = pt7[(pt7['mid_date'] >= starttime) & (pt7['mid_date'] <= endttime)]
v_winter_pt8_2020 = pt8[(pt8['mid_date'] >= starttime) & (pt8['mid_date'] <= endttime)]

In [ ]:
#df of winter velocities at each point from november-february 2020/2021
v_winter_2020 = pd.concat([v_winter_pt1_2020, v_winter_pt2_2020, v_winter_pt3_2020, v_winter_pt4_2020, v_winter_pt5_2020, v_winter_pt6_2020, v_winter_pt7_2020, v_winter_pt8_2020])

#plot
velocity_winter_2020 = v_winter_2020['velocity (m/yr)'].values
time_winter_2020 = pd.to_datetime(v_winter_2020['mid_date']).values
plt.figure(figsize=(20, 4))
plt.scatter(time_winter_2020, velocity_winter_2020)
plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
plt.title('Velocity Winter 2020')
plt.ylim(-50, 600)
plt.show()

winter 2021/2022:

In [ ]:
starttime = datetime(2021, 11, 1)
endttime = datetime(2022, 2, 1)

v_winter_pt1_2021 = pt1[(pt1['mid_date'] >= starttime) & (pt1['mid_date'] <= endttime)]
v_winter_pt2_2021 = pt2[(pt2['mid_date'] >= starttime) & (pt2['mid_date'] <= endttime)]
v_winter_pt3_2021 = pt3[(pt3['mid_date'] >= starttime) & (pt3['mid_date'] <= endttime)]
v_winter_pt4_2021 = pt4[(pt4['mid_date'] >= starttime) & (pt4['mid_date'] <= endttime)]
v_winter_pt5_2021 = pt5[(pt5['mid_date'] >= starttime) & (pt5['mid_date'] <= endttime)]
v_winter_pt6_2021 = pt6[(pt6['mid_date'] >= starttime) & (pt6['mid_date'] <= endttime)]
v_winter_pt7_2021 = pt7[(pt7['mid_date'] >= starttime) & (pt7['mid_date'] <= endttime)]
v_winter_pt8_2021 = pt8[(pt8['mid_date'] >= starttime) & (pt8['mid_date'] <= endttime)]

In [ ]:
#df of winter velocities at each point from november-february 2021/2022
v_winter_2021 = pd.concat([v_winter_pt1_2021, v_winter_pt2_2021, v_winter_pt3_2021, v_winter_pt4_2021, v_winter_pt5_2021, v_winter_pt6_2021, v_winter_pt7_2021, v_winter_pt8_2021])

#plot
velocity_winter_2021 = v_winter_2021['velocity (m/yr)'].values
time_winter_2021 = pd.to_datetime(v_winter_2021['mid_date']).values
plt.figure(figsize=(20, 4))
plt.scatter(time_winter_2021, velocity_winter_2021)  # Plot the data
plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
plt.title('Velocity Winter 2021')
plt.ylim(-50, 600)
plt.show()

winter 2022/2023:

In [ ]:
starttime = datetime(2022, 11, 1)
endttime = datetime(2023, 2, 1)

v_winter_pt1_2022 = pt1[(pt1['mid_date'] >= starttime) & (pt1['mid_date'] <= endttime)]
v_winter_pt2_2022 = pt2[(pt2['mid_date'] >= starttime) & (pt2['mid_date'] <= endttime)]
v_winter_pt3_2022 = pt3[(pt3['mid_date'] >= starttime) & (pt3['mid_date'] <= endttime)]
v_winter_pt4_2022 = pt4[(pt4['mid_date'] >= starttime) & (pt4['mid_date'] <= endttime)]
v_winter_pt5_2022 = pt5[(pt5['mid_date'] >= starttime) & (pt5['mid_date'] <= endttime)]
v_winter_pt6_2022 = pt6[(pt6['mid_date'] >= starttime) & (pt6['mid_date'] <= endttime)]
v_winter_pt7_2022 = pt7[(pt7['mid_date'] >= starttime) & (pt7['mid_date'] <= endttime)]
v_winter_pt8_2022 = pt8[(pt8['mid_date'] >= starttime) & (pt8['mid_date'] <= endttime)]

In [ ]:
#df of winter velocities at each point from november-february 2022/2023
v_winter_2022 = pd.concat([v_winter_pt1_2022, v_winter_pt2_2022, v_winter_pt3_2022, v_winter_pt4_2022, v_winter_pt5_2022, v_winter_pt6_2022, v_winter_pt7_2022, v_winter_pt8_2022])

#plot
velocity_winter_2022 = v_winter_2022['velocity (m/yr)'].values
time_winter_2022 = pd.to_datetime(v_winter_2022['mid_date']).values
plt.figure(figsize=(20, 4))
plt.scatter(time_winter_2022, velocity_winter_2022)
plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
plt.title('Velocity Winter 2022')
plt.ylim(-50, 600)
plt.show()

winter 2023/2024:

In [ ]:
starttime = datetime(2023, 11, 1)
endttime = datetime(2024, 2, 1)

v_winter_pt1_2023 = pt1[(pt1['mid_date'] >= starttime) & (pt1['mid_date'] <= endttime)]
v_winter_pt2_2023 = pt2[(pt2['mid_date'] >= starttime) & (pt2['mid_date'] <= endttime)]
v_winter_pt3_2023 = pt3[(pt3['mid_date'] >= starttime) & (pt3['mid_date'] <= endttime)]
v_winter_pt4_2023 = pt4[(pt4['mid_date'] >= starttime) & (pt4['mid_date'] <= endttime)]
v_winter_pt5_2023 = pt5[(pt5['mid_date'] >= starttime) & (pt5['mid_date'] <= endttime)]
v_winter_pt6_2023 = pt6[(pt6['mid_date'] >= starttime) & (pt6['mid_date'] <= endttime)]
v_winter_pt7_2023 = pt7[(pt7['mid_date'] >= starttime) & (pt7['mid_date'] <= endttime)]
v_winter_pt8_2023 = pt8[(pt8['mid_date'] >= starttime) & (pt8['mid_date'] <= endttime)]

In [ ]:
#df of winter velocities at each point from november-february 2023/2024
v_winter_2023 = pd.concat([v_winter_pt1_2023, v_winter_pt2_2023, v_winter_pt3_2023, v_winter_pt4_2023, v_winter_pt5_2023, v_winter_pt6_2023, v_winter_pt7_2023, v_winter_pt8_2023])

#plot
velocity_winter_2023 = v_winter_2023['velocity (m/yr)'].values
time_winter_2023 = pd.to_datetime(v_winter_2023['mid_date']).values
plt.figure(figsize=(20, 4))
plt.scatter(time_winter_2023, velocity_winter_2023)
plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
plt.title('Velocity Winter 2023')
plt.ylim(-50, 600)
plt.show()

winter 2024/2025:

In [ ]:
starttime = datetime(2024, 11, 1)
endttime = datetime(2025, 2, 1)

v_winter_pt1_2024 = pt1[(pt1['mid_date'] >= starttime) & (pt1['mid_date'] <= endttime)]
v_winter_pt2_2024 = pt2[(pt2['mid_date'] >= starttime) & (pt2['mid_date'] <= endttime)]
v_winter_pt3_2024 = pt3[(pt3['mid_date'] >= starttime) & (pt3['mid_date'] <= endttime)]
v_winter_pt4_2024 = pt4[(pt4['mid_date'] >= starttime) & (pt4['mid_date'] <= endttime)]
v_winter_pt5_2024 = pt5[(pt5['mid_date'] >= starttime) & (pt5['mid_date'] <= endttime)]
v_winter_pt6_2024 = pt6[(pt6['mid_date'] >= starttime) & (pt6['mid_date'] <= endttime)]
v_winter_pt7_2024 = pt7[(pt7['mid_date'] >= starttime) & (pt7['mid_date'] <= endttime)]
v_winter_pt8_2024 = pt8[(pt8['mid_date'] >= starttime) & (pt8['mid_date'] <= endttime)]

In [ ]:
#df of winter velocities at each point from november-february 2024/2025
v_winter_2024 = pd.concat([v_winter_pt1_2024, v_winter_pt2_2024, v_winter_pt3_2024, v_winter_pt4_2024, v_winter_pt5_2024, v_winter_pt6_2024, v_winter_pt7_2024, v_winter_pt8_2024])

#plot
velocity_winter_2024 = v_winter_2024['velocity (m/yr)'].values
time_winter_2024 = pd.to_datetime(v_winter_2024['mid_date']).values
plt.figure(figsize=(20, 4))
plt.scatter(time_winter_2024, velocity_winter_2024)
plt.xlabel('Time')
plt.ylabel('Velocity (m/yr)')
plt.title('Velocity Winter 2024')
plt.ylim(-50, 1000)
plt.show()

accumulation zone velocities: (points 5,6,7,8)